First, let's import the os package and list the solutions/downloaded directory to see which files we have:

In [1]:
import os
os.listdir('solutions/downloaded')

['USDCHF.json',
 'USDCAD.json',
 'GOOG.csv',
 'NZDUSD.json',
 'SHLD.csv',
 'EURUSD.json',
 'AMZN.csv',
 'USDJPY.json',
 'TSLA.csv',
 'GBPUSD.json',
 'AAPL.csv',
 'AUDUSD.json']

Now we can import the json and pandas libraries, and try to load one of the Forex files into a dataframe, inspecting the head() of the frame..

In [2]:
import json
import pandas as pd
with open( 'solutions/downloaded/EURUSD.json','r') as handle:
    eurusd_json = json.load( handle )

eurusd_df = pd.DataFrame( eurusd_json )
print( eurusd_df.head() )

    Close    DateTime    High     Low    Open
0  1.2339  1523836800   1.234  1.2331  1.2332
1  1.2336  1523840400  1.2343  1.2334  1.2340
2   1.234  1523844000  1.2342  1.2334  1.2336
3  1.2338  1523847600  1.2341  1.2338  1.2340
4  1.2331  1523851200  1.2338  1.2331  1.2338


Looks pretty reasonable, so now we can use dtypes to print the python 'type' associated with each column.

In [3]:
print( eurusd_df.dtypes )

Close        object
DateTime      int64
High         object
Low          object
Open        float64
dtype: object


Uh oh. Our Open prices are float64s which looks good - but why are our other columns High, Low and Close all 'objects'?
We can use pandas .astype() method on a series to convert it to float64s. We have to import numpy first as that's where float64 is defined:

In [4]:
import numpy as np
try:
    eurusd_df['High'] = eurusd_df['High'].astype(np.float64)
except ValueError as e:
    print( e )

could not convert string to float: 'profit'


Aha! So somewhere in our 'High' column we have the word 'profit' as a string, which has caused pandas to coerce the entire column as strings. So what do we do about this?
First, let's create a pandas Series, called 'high' which will be our 'High' column, but converted to numbers. If we use errors='coerce' it will fill bad values with NaN:

In [5]:
high = pd.to_numeric( eurusd_df.High, errors = 'coerce' )

Then we can create a boolean map to find the NaN values with isna(), and then apply that index to the dataframe as a filter to find bad values:

In [6]:
high_bad_index = high.isna()
print( eurusd_df[high_bad_index] )

      Close    DateTime    High     Low    Open
247  1.2119  1525071600  profit  1.2109  1.2137


Ok, so it's just one row. This isn't so bad. So that column we created earlier, 'high', we know that is all proper numbers now, except one NaN, so let's replace the 'High' series in our dataframe with that converted series and then check dtypes again:

In [7]:
eurusd_df['High'] = high
print(eurusd_df.dtypes)

Close        object
DateTime      int64
High        float64
Low          object
Open        float64
dtype: object


So what about writing a general function to do this for us, whereby we pass it a dataframe and a column name we want forced numeric? Let's combine the steps we had above to do that for us:

In [8]:
def make_numeric( df, col_name ):
    ''' takes a dataframe and a column name and makes the column numeric, setting bad values to NaN '''
    series_as_numbers = pd.to_numeric( df[ col_name ], errors='coerce' )
    bad_index = series_as_numbers.isna()
    print( df[ bad_index ] )
    df[ col_name ] = series_as_numbers
    return df

eurusd_df = make_numeric( eurusd_df, 'Low' )
print( eurusd_df.dtypes )

      Close    DateTime    High   Low    Open
120  1.2271  1524441600  1.2279  &&&&  1.2274
Close        object
DateTime      int64
High        float64
Low         float64
Open        float64
dtype: object


So! There was one low price set to &&&&, not too bad, let's pass in the final price column, 'Close':

In [9]:
eurusd_df = make_numeric( eurusd_df, 'Close' )
print( eurusd_df.dtypes )

   Close    DateTime    High     Low    Open
40  None  1523980800  1.2357  1.2338  1.2345
Close       float64
DateTime      int64
High        float64
Low         float64
Open        float64
dtype: object


Now that all our numbers are correctly typed as numbers, the real power of dataframes can be exploited. First let's fix up this DateTime, it's currenty a large integer specifying the number of seconds since the unix epoch (Midnight January 1st 1970). 

In [10]:
from datetime import datetime, timezone
print(eurusd_df.head())

    Close    DateTime    High     Low    Open
0  1.2339  1523836800  1.2340  1.2331  1.2332
1  1.2336  1523840400  1.2343  1.2334  1.2340
2  1.2340  1523844000  1.2342  1.2334  1.2336
3  1.2338  1523847600  1.2341  1.2338  1.2340
4  1.2331  1523851200  1.2338  1.2331  1.2338


We can use a list comprehension to create a new index with the values of the DateTime column passed to datetime.fromtimestamp():

In [11]:
eurusd_df.index = [ datetime.fromtimestamp( x, timezone.utc ) for x in eurusd_df['DateTime'] ]
print( eurusd_df.head() )

                            Close    DateTime    High     Low    Open
2018-04-16 00:00:00+00:00  1.2339  1523836800  1.2340  1.2331  1.2332
2018-04-16 01:00:00+00:00  1.2336  1523840400  1.2343  1.2334  1.2340
2018-04-16 02:00:00+00:00  1.2340  1523844000  1.2342  1.2334  1.2336
2018-04-16 03:00:00+00:00  1.2338  1523847600  1.2341  1.2338  1.2340
2018-04-16 04:00:00+00:00  1.2331  1523851200  1.2338  1.2331  1.2338


Beautiful! We have a dataframe of all floating point pricings with an index properly cast as datetime objects. Now armed with this cleaner data we can go ahead and see if we can create a plot. Pandas is tightly integrated with matplotlib

In [ ]:
import matplotlib.pyplot as plt
ax = plt.gca() # gca means get current axis

eurusd_df.plot(kind='line',x='DateTime', y='High', color='green', ax=ax)
eurusd_df.plot(kind='line',x='DateTime', y='Low', color='red', ax=ax)

plt.show()